In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import openml
import os
import pandas as pd
import math

import sys
sys.path.append("../..")
from src.load_datasets import load_dataset, load_rankings, load_train_data
import src.evaluate_regression

# Load data

In [ ]:
X_train, y_train = load_train_data('../../data/raw/dataset_rank_train.csv')

# Rebuild Federicos DT

In [68]:
def my_custom_cross_validated_indices(df: pd.DataFrame, factors, target: str,
                                   **kfoldargs):
    df_factors = df.groupby(factors)[target].mean().reset_index()
    X_factors, y_factors = df_factors.drop(target, axis=1), df_factors[target]

    indices = []
    for itr, ite in KFold(**kfoldargs).split(X_factors, y_factors):
        tr = pd.merge(X_factors.iloc[itr], df.reset_index(), on=factors)['index']  # "index" is the index of df
        te = pd.merge(X_factors.iloc[ite], df.reset_index(), on=factors)['index']  # "index" is the index of df
        indices.append([tr, te])

    return indices

In [93]:
from category_encoders import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor

# Load data
X_train, y_train = load_train_data('../../data/raw/dataset_rank_train.csv')
X_train = X_train.drop("rank", axis=1)

# Define variables for ranking
factors = ["dataset", "model", "tuning", "scoring"]
new_index = "encoder"
target = "cv_score"

# Get indices for CV
#indices = src.evaluate_regression.custom_cross_validated_indices(pd.concat([X_train, y_train], axis=1), factors, target, n_splits=5, shuffle=True, random_state=1444)
indices = my_custom_cross_validated_indices(pd.concat([X_train, y_train], axis=1), factors, target, n_splits=5, shuffle=True, random_state=1444)

print(X_train.head())
print(X_train.shape)

for fold in indices:
    # Define data of current fold
    X_train_fold = X_train.iloc[fold[0]]
    X_test_fold = X_train.iloc[fold[1]]
    y_train_fold = y_train.iloc[fold[0]]
    y_test_fold = y_train.iloc[fold[1]]
    
    # Train model and predict
    dummy_pipe = Pipeline([("encoder", OneHotEncoder()), ("model", DecisionTreeRegressor())])
    y_pred = pd.Series(dummy_pipe.fit(X_train_fold, y_train_fold).predict(X_test_fold), index=y_test_fold.index, name="cv_score_pred")
    df_pred = pd.concat([X_test_fold, y_test_fold, y_pred], axis=1)
    
    # Rankings and avg_spearman
    rankings_test = src.evaluate_regression.get_rankings(df_pred, factors=factors, new_index=new_index, target="cv_score")
    rankings_pred = src.evaluate_regression.get_rankings(df_pred, factors=factors, new_index=new_index, target="cv_score_pred")
    print(src.evaluate_regression.average_spearman(rankings_test, rankings_pred))

Loading train data from '../../data/raw/dataset_rank_train.csv'...
   dataset model tuning scoring      encoder
0     1169   KNC  model     ACC  BUCV2RGLMME
1     1169   KNC  model     ACC      BUCV2TE
2     1169   KNC  model     ACC          CBE
3     1169   KNC  model     ACC           CE
4     1169   KNC  model     ACC   CV10RGLMME
(36054, 5)


IndexError: positional indexers are out-of-bounds

In [5]:
print(X_train.shape)
X_train, X_test, y_train, y_test = src.evaluate_regression.custom_train_test_split(pd.concat([X_train, y_train], axis=1), factors, target)
print(X_train.shape)

# Train model and predict
dummy_pipe = Pipeline([("encoder", OneHotEncoder()), ("model", DecisionTreeRegressor())])
y_pred = pd.Series(dummy_pipe.fit(X_train, y_train).predict(X_test), index=y_test.index, name="cv_score_pred")
df_pred = pd.concat([X_test, y_test, y_pred], axis=1)

# Rankings and avg_spearman
rankings_test = src.evaluate_regression.get_rankings(df_pred, factors=factors, new_index=new_index, target="cv_score")
rankings_pred = src.evaluate_regression.get_rankings(df_pred, factors=factors, new_index=new_index, target="cv_score_pred")
print(src.evaluate_regression.average_spearman(rankings_test, rankings_pred))

(36054, 5)
(26989, 5)
0.5039204789206002


### Analyze the custom indices function

In [ ]:
#def custom_cross_validated_indices(df: pd.DataFrame, factors: Iterable[str], target: str,
#                                   **kfoldargs) -> List[List[Iterable[int]]]:
#    df_factors = df.groupby(factors)[target].mean().reset_index()
#    X_factors, y_factors = df_factors.drop(target, axis=1), df_factors[target]

#    indices = []
#    for itr, ite in KFold(**kfoldargs).split(X_factors, y_factors):
#        tr = pd.merge(X_factors.iloc[itr], df.reset_index(), on=factors).index  # "index" is the index of df
#        te = pd.merge(X_factors.iloc[ite], df.reset_index(), on=factors).index  # "index" is the index of df
#        indices.append([tr, te])

#    return indices

In [15]:
# Load data
X_train, y_train = load_train_data('../../data/raw/dataset_rank_train.csv')
X_train = X_train.drop("rank", axis=1)
df = pd.concat([X_train, y_train], axis=1)
print(df.shape)
df.head()

Loading train data from '../../data/raw/dataset_rank_train.csv'...
(36054, 6)


,dataset,model,tuning,scoring,encoder,cv_score
0,1169,KNC,model,ACC,BUCV2RGLMME,0.650562
1,1169,KNC,model,ACC,BUCV2TE,0.650712
2,1169,KNC,model,ACC,CBE,0.626492
3,1169,KNC,model,ACC,CE,0.625839
4,1169,KNC,model,ACC,CV10RGLMME,0.652093


In [17]:
factors = ["dataset", "model", "tuning", "scoring"]
new_index = "encoder"
target = "cv_score"

In [18]:
from sklearn.model_selection import KFold

df_factors = df.groupby(factors)[target].mean().reset_index()
df_factors

,dataset,model,tuning,scoring,cv_score
0,3,DTC,full,ACC,0.884794
1,3,DTC,full,AUC,0.882987
2,3,DTC,full,F1,0.894308
3,3,DTC,model,AUC,0.938125
4,3,DTC,model,F1,0.942405
...,...,...,...,...,...
1156,43922,LR,no,F1,0.866286
1157,43922,SVC,full,ACC,0.845074
1158,43922,SVC,no,ACC,0.865959
1159,43922,SVC,no,AUC,0.867327


In [21]:
df_factors = df.groupby(factors)[target].mean().reset_index()
X_factors, y_factors = df_factors.drop(target, axis=1), df_factors[target]

In [24]:
folds = KFold(n_splits=5, shuffle=True, random_state=1444).split(X_factors, y_factors)
itr, ite = next(folds)

In [27]:
for x in ite:
    if x in itr:
        print(x)

In [31]:
tr_0 = pd.merge(X_factors.iloc[itr], df.reset_index(), on=factors).index  # "index" is the index of df
te_0 = pd.merge(X_factors.iloc[ite], df.reset_index(), on=factors).index  # "index" is the index of df

In [33]:
print(tr_0)
print(te_0)

RangeIndex(start=0, stop=28876, step=1)
RangeIndex(start=0, stop=7178, step=1)


In [36]:
itr[:20]

array([ 0,  1,  2,  3,  4,  5,  6,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       19, 20, 21])

In [39]:
# Next iteration
itr, ite = next(folds)

In [40]:
itr[:20]

array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13, 14, 15, 16, 17, 18,
       19, 20, 21])

In [41]:
tr_1 = pd.merge(X_factors.iloc[itr], df.reset_index(), on=factors).index  # "index" is the index of df
te_1 = pd.merge(X_factors.iloc[ite], df.reset_index(), on=factors).index  # "index" is the index of df
print(tr_1)
print(te_1)

RangeIndex(start=0, stop=28910, step=1)
RangeIndex(start=0, stop=7144, step=1)


In [ ]:
df_factors = df.groupby(factors)[target].mean().reset_index()
X_factors, y_factors = df_factors.drop(target, axis=1), df_factors[target]

indices = []
for itr, ite in KFold(n_splits=5, shuffle=True, random_state=1444).split(X_factors, y_factors):
    tr = pd.merge(X_factors.iloc[itr], df.reset_index(), on=factors).index  # "index" is the index of df
    te = pd.merge(X_factors.iloc[ite], df.reset_index(), on=factors).index  # "index" is the index of df
    indices.append([tr, te])

#return indices

In [42]:
indices = src.evaluate_regression.custom_cross_validated_indices(pd.concat([X_train, y_train], axis=1), factors, target, n_splits=5, shuffle=True, random_state=1444)

In [ ]:
fold_0 = indices[0]
fold_1 = indices[1]
fold_2 = indices[2]
fold_3 = indices[3]
fold_4 = indices[4]

In [47]:
import numpy as np 

# Calculate overlap of indices between different folds
for i in range(5):
    if i != 4:
        for j in range(i+1, 5):
            print(f"Number of same indices of fold {i} and {j}")
            
            # Train set
            print(f"Train set")
            print(f"Number of indices in fold {i}: {len(indices[i][0])}")
            print(f"Number of indices in fold {j}: {len(indices[j][0])}")
            
            intersect = np.intersect1d(indices[i][0], indices[j][0])
            print(f"Number of equal indices: {len(intersect)}")
            
            
            # Test set
            print(f"Test set")
            print(f"Number of indices in fold {i}: {len(indices[i][1])}")
            print(f"Number of indices in fold {j}: {len(indices[j][1])}")
            
            intersect = np.intersect1d(indices[i][1], indices[j][1])
            print(f"Number of equal indices: {len(intersect)}")

Number of same indices of fold 0 and 1
Train set
Number of indices in fold 0: 28876
Number of indices in fold 1: 28807
Number of equal indices: 28807
Test set
Number of indices in fold 0: 7178
Number of indices in fold 1: 7247
Number of equal indices: 7178
Number of same indices of fold 0 and 2
Train set
Number of indices in fold 0: 28876
Number of indices in fold 2: 28910
Number of equal indices: 28876
Test set
Number of indices in fold 0: 7178
Number of indices in fold 2: 7144
Number of equal indices: 7144
Number of same indices of fold 0 and 3
Train set
Number of indices in fold 0: 28876
Number of indices in fold 3: 28805
Number of equal indices: 28805
Test set
Number of indices in fold 0: 7178
Number of indices in fold 3: 7249
Number of equal indices: 7178
Number of same indices of fold 0 and 4
Train set
Number of indices in fold 0: 28876
Number of indices in fold 4: 28818
Number of equal indices: 28818
Test set
Number of indices in fold 0: 7178
Number of indices in fold 4: 7236
Nu

In [50]:
import numpy as np 

indices = src.evaluate_regression.custom_cross_validated_indices(pd.concat([X_train, y_train], axis=1), factors, target, n_splits=5, shuffle=True, random_state=1444)

# Calculate overlap of train and test per fold
for i in range(5):
    # Train set
    print(f"Fold: {i}")
    print(f"Number of indices in train: {len(indices[i][0])}")
    print(f"Number of indices in test : {len(indices[i][1])}")
    
    
    intersect = np.intersect1d(indices[i][0], indices[i][1])
    print(f"Number of equal indices: {len(intersect)}")

Fold: 0
Number of indices in train: 28876
Number of indices in test : 7178
Number of equal indices: 7178
Fold: 1
Number of indices in train: 28807
Number of indices in test : 7247
Number of equal indices: 7247
Fold: 2
Number of indices in train: 28910
Number of indices in test : 7144
Number of equal indices: 7144
Fold: 3
Number of indices in train: 28805
Number of indices in test : 7249
Number of equal indices: 7249
Fold: 4
Number of indices in train: 28818
Number of indices in test : 7236
Number of equal indices: 7236


In [51]:
def validate_5_fold_indices(indices):
    for i in range(5):
        # Train set
        print(f"Fold: {i}")
        print(f"Number of indices in train: {len(indices[i][0])}")
        print(f"Number of indices in test : {len(indices[i][1])}")


        intersect = np.intersect1d(indices[i][0], indices[i][1])
        print(f"Number of equal indices: {len(intersect)}")

In [83]:
def my_custom_cross_validated_indices(df: pd.DataFrame, factors, target: str,
                                   **kfoldargs):
    #df.index = df.index.tolist()
    df_factors = df.groupby(factors)[target].mean().reset_index()
    X_factors, y_factors = df_factors.drop(target, axis=1), df_factors[target]
    
    indices = []
    for itr, ite in KFold(**kfoldargs).split(X_factors, y_factors):
        tr = pd.merge(X_factors.iloc[itr], df.reset_index(), on=factors)['index']  # "index" is the index of df
        te = pd.merge(X_factors.iloc[ite], df.reset_index(), on=factors)['index']  # "index" is the index of df
        indices.append([tr, te])

    return indices

In [84]:
my_indices = my_custom_cross_validated_indices(pd.concat([X_train, y_train], axis=1), factors, target, n_splits=5, shuffle=True, random_state=1444)
validate_5_fold_indices(my_indices)

Fold: 0
Number of indices in train: 28876
Number of indices in test : 7178
Number of equal indices: 0
Fold: 1
Number of indices in train: 28807
Number of indices in test : 7247
Number of equal indices: 0
Fold: 2
Number of indices in train: 28910
Number of indices in test : 7144
Number of equal indices: 0
Fold: 3
Number of indices in train: 28805
Number of indices in test : 7249
Number of equal indices: 0
Fold: 4
Number of indices in train: 28818
Number of indices in test : 7236
Number of equal indices: 0


In [85]:
my_indices[0]

[0        16858
 1        16859
 2        16860
 3        16861
 4        16862
          ...  
 28871     6434
 28872     6435
 28873     6436
 28874     6437
 28875     6438
 Name: index, Length: 28876, dtype: int64,
 0        6502
 1        6503
 2        6504
 3        6505
 4        6506
         ...  
 7173    24563
 7174    24564
 7175    24565
 7176    24566
 7177    24567
 Name: index, Length: 7178, dtype: int64]

In [63]:
tmp = df.reset_index()
tmp.index

RangeIndex(start=0, stop=36054, step=1)

In [61]:
tmp['index'].dtype

dtype('int64')

In [70]:
indices = src.evaluate_regression.custom_cross_validated_indices(pd.concat([X_train, y_train], axis=1), factors, target, n_splits=5, shuffle=True, random_state=1444)
print(indices[0][0])
print(indices[0][1])

RangeIndex(start=0, stop=28876, step=1)
RangeIndex(start=0, stop=7178, step=1)


In [71]:
indices = my_custom_cross_validated_indices(pd.concat([X_train, y_train], axis=1), factors, target, n_splits=5, shuffle=True, random_state=1444)
print(indices[0][0])
print(indices[0][1])

0        16858
1        16859
2        16860
3        16861
4        16862
         ...  
28871     6434
28872     6435
28873     6436
28874     6437
28875     6438
Name: index, Length: 28876, dtype: int64
0        6502
1        6503
2        6504
3        6505
4        6506
        ...  
7173    24563
7174    24564
7175    24565
7176    24566
7177    24567
Name: index, Length: 7178, dtype: int64


In [72]:
X_train.index

RangeIndex(start=0, stop=36054, step=1)

In [73]:
tmp = X_train.copy()

In [75]:
tmp.index.tolist()

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [76]:
tmp.index

RangeIndex(start=0, stop=36054, step=1)

In [77]:
tmp.index = tmp.index.tolist()
tmp.index

Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,
       ...
       36044, 36045, 36046, 36047, 36048, 36049, 36050, 36051, 36052, 36053],
      dtype='int64', length=36054)

In [88]:
maximum = 0
for f in range(5):
    for i in range(2):
        if max(my_indices[f][i]) > maximum:
            maximum = max(my_indices[f][i])

print(maximum)

36053


In [89]:
X_train.shape

(36054, 5)

### Evaluate get_rankings

In [148]:
def my_get_rankings(df: pd.DataFrame, factors, new_index, target) -> pd.DataFrame:

    assert set(factors).issubset(df.columns)

    rankings = {}
    #print(df.shape)
    for group, indices in df.groupby(factors).groups.items():
        #print(indices)
        #print(df.shape)
        score = df.loc[indices].set_index(new_index)[target]
        rankings[group] = src.evaluate_regression.score2ranking(score, ascending=False)

    return pd.DataFrame(rankings)

In [149]:
from category_encoders import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor

# Load data
X_train, y_train = load_train_data('../../data/raw/dataset_rank_train.csv')
X_train = X_train.drop("rank", axis=1)

# Define variables for ranking
factors = ["dataset", "model", "tuning", "scoring"]
new_index = "encoder"
target = "cv_score"

# Get indices for CV
#indices = src.evaluate_regression.custom_cross_validated_indices(pd.concat([X_train, y_train], axis=1), factors, target, n_splits=5, shuffle=True, random_state=1444)
indices = my_custom_cross_validated_indices(pd.concat([X_train, y_train], axis=1), factors, target, n_splits=5, shuffle=True, random_state=1444)

#print(X_train.head())
#print(X_train.shape)

for fold in indices:
    # Define data of current fold
    X_train_fold = X_train.iloc[fold[0]]
    X_test_fold = X_train.iloc[fold[1]]
    y_train_fold = y_train.iloc[fold[0]]
    y_test_fold = y_train.iloc[fold[1]]
    
    # Train model and predict
    dummy_pipe = Pipeline([("encoder", OneHotEncoder()), ("model", DecisionTreeRegressor())])
    y_pred = pd.Series(dummy_pipe.fit(X_train_fold, y_train_fold).predict(X_test_fold), index=y_test_fold.index, name="cv_score_pred")
    df_pred = pd.concat([X_test_fold, y_test_fold, y_pred], axis=1)
    
    # Rankings and avg_spearman
    #rankings_test = src.evaluate_regression.get_rankings(df_pred, factors=factors, new_index=new_index, target="cv_score")
    rankings_test = my_get_rankings(df_pred, factors=factors, new_index=new_index, target="cv_score")
    #rankings_pred = src.evaluate_regression.get_rankings(df_pred, factors=factors, new_index=new_index, target="cv_score_pred")
    rankings_pred = my_get_rankings(df_pred, factors=factors, new_index=new_index, target="cv_score_pred")
    print(src.evaluate_regression.average_spearman(rankings_test, rankings_pred))

Loading train data from '../../data/raw/dataset_rank_train.csv'...
0.3968434938878512
0.4992000911213593
0.4919303066361779
0.49113509595746835
0.4964681520843705


In [115]:
rankings = {}
for group, indices in df.groupby(factors).groups.items():
    score = df.iloc[indices].set_index(new_index)[target]
    rankings[group] = src.evaluate_regression.score2ranking(score, ascending=False)

In [ ]:
# group: Contains the fixed context
# indices: Indices of the context + encoder (all)

In [116]:
group

(43922, 'SVC', 'no', 'F1')

In [117]:
indices

Index([6407, 6408, 6409, 6410, 6411, 6412, 6413, 6414, 6415, 6416, 6417, 6418,
       6419, 6420, 6421, 6422, 6423, 6424, 6425, 6426, 6427, 6428, 6429, 6430,
       6431, 6432, 6433, 6434, 6435, 6436, 6437, 6438],
      dtype='int64')

In [120]:
df.iloc[[6407, 6408, 6409, 6410, 6411]]

,dataset,model,tuning,scoring,encoder,cv_score
6407,43922,SVC,no,F1,BE,0.906552
6408,43922,SVC,no,F1,BUCV10RGLMME,0.891730
6409,43922,SVC,no,F1,BUCV10TE,0.893247
6410,43922,SVC,no,F1,BUCV2RGLMME,0.896300
6411,43922,SVC,no,F1,BUCV2TE,0.896304


In [140]:
df_pred.groupby(factors).groups.get((3, 'KNC', 'full', 'F1'))

Index([6502, 6503, 6504, 6505, 6506, 6507, 6508, 6509, 6510, 6511, 6512, 6513,
       6514, 6515, 6516, 6517, 6518, 6519, 6520, 6521, 6522, 6523, 6524, 6525,
       6526, 6527, 6528, 6529, 6530, 6531, 6532, 6533],
      dtype='int64')

In [142]:
df_pred.groupby(factors).groups.get((3, 'LR', 'model', 'ACC'))

Index([21878, 21879, 21880, 21881, 21882, 21883, 21884, 21885, 21886, 21887,
       21888, 21889, 21890, 21891, 21892, 21893, 21894, 21895, 21896, 21897,
       21898, 21899, 21900, 21901, 21902, 21903, 21904, 21905, 21906, 21907,
       21908, 21909],
      dtype='int64')

In [143]:
df_pred

,dataset,model,tuning,scoring,encoder,cv_score,cv_score_pred
6502,3,KNC,full,F1,BE,0.793204,0.794576
6503,3,KNC,full,F1,BUCV10RGLMME,0.929585,0.922319
6504,3,KNC,full,F1,BUCV10TE,0.931987,0.926679
6505,3,KNC,full,F1,BUCV2RGLMME,0.929742,0.922397
6506,3,KNC,full,F1,BUCV2TE,0.931919,0.926488
...,...,...,...,...,...,...,...
24563,43922,LGBMC,no,AUC,PBTE01,0.947976,0.893167
24564,43922,LGBMC,no,AUC,RGLMME,0.892612,0.871168
24565,43922,LGBMC,no,AUC,SE,0.926384,0.930628
24566,43922,LGBMC,no,AUC,TE,0.892612,0.871551
